In [74]:
import string
import numpy as np
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\01din\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
#Change paths sorry
private_scrape = pd.read_excel("C:/Users/01din/Desktop/data APG/scrape_results/private_scrape_results.xlsx")
public_scrape = pd.read_excel("C:/Users/01din/Desktop/data APG/scrape_results/public_scrape_results.xlsx")

In [1]:
from nltk.corpus import stopwords
#Gettings document values, and replace nan values with a space
private_corpus = private_scrape.paragraph.fillna(' ')
public_corpus = public_scrape.paragraph.fillna(' ')
#Combine into 1 list
corpus = private_corpus + public_corpus
#Words that need to be removed, consist of stopwords in dutch and the months
#TODO: some docs are in english
dutch_stopwords = stopwords.words('dutch')
months = ['Januari', "Februari", "April", "Mei", "Juni", "Juli", "Augustus", "September", "Oktober", "November", "December"]
words_to_remove = dutch_stopwords + months

NameError: name 'private_scrape' is not defined

In [102]:
#Pass the docsuments and the words that must be removed
def clean_documents(corpus, words_to_remove):
    for i in range(len(corpus)):
        #Retrieve the doc and convert to str (some issues with NANs)
        doc = str(corpus[i])
        #Split into list of words
        words = doc.split()
        cleaned_doc = []
        #Remove words in words_to_remove set
        for word in words:
            if word not in words_to_remove:
                cleaned_doc.append(word)
        #Join words list back into one string
        cleaned_doc = " ".join(cleaned_doc)
        #Remove punctuation
        cleaned_doc = cleaned_doc.translate(str.maketrans("", "", string.punctuation))
        #Remove numbers
        cleaned_doc = "".join([i for i in cleaned_doc if not i.isdigit()])
        #Remove double spaces
        while "  " in cleaned_doc:
            cleaned_doc = cleaned_doc.replace("  ", " ")
        #Reassign
        corpus[i] = cleaned_doc
    return corpus
clean_corpus = clean_documents(corpus, words_to_remove)

In [61]:
#This max_df value seems not to give the wished results, if max_df=0.3, only words that appear in <30% of all docs should be considered
vectorizer = TfidfVectorizer(max_df = 1.0)
vec_trained = vectorizer.fit_transform(clean_corpus.astype("U"))

In [62]:
vectorizer.get_feature_names_out()

array(['00', '000', '01', ..., 'én', 'één', 'óf'], dtype=object)

In [63]:
vec_trained.shape

(1375, 7484)